In [1]:
import sys
import numpy as np
import scipy.stats
import operator
import itertools
import math

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

from scipy.stats import ttest_ind

import multiprocessing

import random

"""
random.seed(int(sys.argv[1]))
np.random.seed(int(sys.argv[1]))
"""

np.seterr(divide='ignore')

def selAutomaticEpsilonLexicase(individuals, k):
    """
    Returns an individual that does the best on the fitness cases when considered one at a
    time in random order. 
    https://push-language.hampshire.edu/uploads/default/original/1X/35c30e47ef6323a0a949402914453f277fb1b5b0.pdf
    Implemented lambda_epsilon_y implementation.
    :param individuals: A list of individuals to select from.
    :param k: The number of individuals to select.
    :returns: A list of selected individuals.
    """      
    selected_individuals = [] 
    individual_complexities = [np.sum([type(component) == gp.Primitive for component in individual])
                               for individual in individuals]
    
    for i in range(k):
        fit_weights = individuals[0].fitness.weights
        
        random_individual_complexity = random.choice(individual_complexities)
        candidates = []
        for individual_num, individual in enumerate(individuals):
            if individual_complexities[individual_num] <= random_individual_complexity:
                candidates.append(individual)
        cases = list(range(len(individuals[0].fitness.values)))
        random.shuffle(cases)

        while len(cases) > 0 and len(candidates) > 1: 
            errors_for_this_case = [x.fitness.values[cases[0]] for x in candidates]
            median_val = np.median(errors_for_this_case)
            median_absolute_deviation = np.median([abs(x - median_val) for x in errors_for_this_case])
            if fit_weights[cases[0]] > 0:
                best_val_for_case = max(errors_for_this_case) 
                min_val_to_survive = best_val_for_case - median_absolute_deviation
                candidates = list(filter(lambda x: x.fitness.values[cases[0]] >= min_val_to_survive, candidates))
            else:
                best_val_for_case = min(errors_for_this_case) 
                max_val_to_survive = best_val_for_case + median_absolute_deviation
                candidates = list(filter(lambda x: x.fitness.values[cases[0]] <= max_val_to_survive, candidates))
            
            cases.pop(0)
                     
        selected_individuals.append(random.choice(candidates))
    
    return selected_individuals


num_splits = 24

#def runGA():
# Generate the distributions to be used in the GP optimization process

# Index by: [sample number (0-29)]
sig_diff_samples_0v10 = np.random.normal(loc=0, scale=3, size=(30, 500))
sig_diff_samples_10v0 = np.random.normal(loc=10, scale=5, size=(30, 500))

sig_diff_samples_0v10_same_std = np.random.normal(loc=0, scale=3, size=(30, 500))
sig_diff_samples_10v0_same_std = np.random.normal(loc=100, scale=3, size=(30, 500))

sig_diff_samples_0v100 = np.random.normal(loc=0, scale=3, size=(30, 500))
sig_diff_samples_100v0 = np.random.normal(loc=100, scale=5, size=(30, 500))

sig_diff_samples_0v100_same_std = np.random.normal(loc=0, scale=30, size=(30, 500))
sig_diff_samples_100v0_same_std = np.random.normal(loc=100, scale=30, size=(30, 500))

sig_diff_samples_0v1000 = np.random.normal(loc=0, scale=3, size=(30, 500))
sig_diff_samples_1000v0 = np.random.normal(loc=1000, scale=5, size=(30, 500))

sig_diff_samples_0v1000_same_std = np.random.normal(loc=0, scale=300, size=(30, 500))
sig_diff_samples_1000v0_same_std = np.random.normal(loc=1000, scale=300, size=(30, 500))

# Index by: [group (0/1)][sample number (0-29)]
both_dist = np.append(sig_diff_samples_0v10.reshape(
                          sig_diff_samples_0v10.shape[0] * sig_diff_samples_0v10.shape[1]),
                      sig_diff_samples_10v0.reshape(
                          sig_diff_samples_10v0.shape[0] * sig_diff_samples_10v0.shape[1]))
both_dist_mean = np.mean(both_dist)
both_dist_sd = np.std(both_dist)
permuted_samples_0v10 = np.random.normal(loc=both_dist_mean, scale=both_dist_sd, size=(2, 30, 500))
null_samples_0v10 = np.random.normal(loc=both_dist_mean, scale=both_dist_sd, size=(2, 30, 500))

both_dist = np.append(sig_diff_samples_0v10_same_std.reshape(
                          sig_diff_samples_0v10_same_std.shape[0] * sig_diff_samples_0v10_same_std.shape[1]),
                      sig_diff_samples_10v0_same_std.reshape(
                          sig_diff_samples_10v0_same_std.shape[0] * sig_diff_samples_10v0_same_std.shape[1]))
both_dist_mean = np.mean(both_dist)
both_dist_sd = np.std(both_dist)
permuted_samples_0v10_same_std = np.random.normal(loc=both_dist_mean, scale=both_dist_sd, size=(2, 30, 500))
null_samples_0v10_same_std = np.random.normal(loc=both_dist_mean, scale=both_dist_sd, size=(2, 30, 500))

both_dist = np.append(sig_diff_samples_0v100.reshape(
                          sig_diff_samples_0v100.shape[0] * sig_diff_samples_0v100.shape[1]),
                      sig_diff_samples_100v0.reshape(
                          sig_diff_samples_100v0.shape[0] * sig_diff_samples_100v0.shape[1]))
both_dist_mean = np.mean(both_dist)
both_dist_sd = np.std(both_dist)
permuted_samples_0v100 = np.random.normal(loc=both_dist_mean, scale=both_dist_sd, size=(2, 30, 500))
null_samples_0v100 = np.random.normal(loc=both_dist_mean, scale=both_dist_sd, size=(2, 30, 500))

both_dist = np.append(sig_diff_samples_0v100_same_std.reshape(
                          sig_diff_samples_0v100_same_std.shape[0] * sig_diff_samples_0v100_same_std.shape[1]),
                      sig_diff_samples_100v0_same_std.reshape(
                          sig_diff_samples_100v0_same_std.shape[0] * sig_diff_samples_100v0_same_std.shape[1]))
both_dist_mean = np.mean(both_dist)
both_dist_sd = np.std(both_dist)
permuted_samples_0v100_same_std = np.random.normal(loc=both_dist_mean, scale=both_dist_sd, size=(2, 30, 500))
null_samples_0v100_same_std = np.random.normal(loc=both_dist_mean, scale=both_dist_sd, size=(2, 30, 500))

both_dist = np.append(sig_diff_samples_0v1000.reshape(
                          sig_diff_samples_0v1000.shape[0] * sig_diff_samples_0v1000.shape[1]),
                      sig_diff_samples_1000v0.reshape(
                          sig_diff_samples_1000v0.shape[0] * sig_diff_samples_1000v0.shape[1]))
both_dist_mean = np.mean(both_dist)
both_dist_sd = np.std(both_dist)
permuted_samples_0v1000 = np.random.normal(loc=both_dist_mean, scale=both_dist_sd, size=(2, 30, 500))
null_samples_0v1000 = np.random.normal(loc=both_dist_mean, scale=both_dist_sd, size=(2, 30, 500))

both_dist = np.append(sig_diff_samples_0v1000_same_std.reshape(
                          sig_diff_samples_0v1000_same_std.shape[0] * sig_diff_samples_0v1000_same_std.shape[1]),
                      sig_diff_samples_1000v0_same_std.reshape(
                          sig_diff_samples_1000v0_same_std.shape[0] * sig_diff_samples_1000v0_same_std.shape[1]))
both_dist_mean = np.mean(both_dist)
both_dist_sd = np.std(both_dist)
permuted_samples_0v1000_same_std = np.random.normal(loc=both_dist_mean, scale=both_dist_sd, size=(2, 30, 500))
null_samples_0v1000_same_std = np.random.normal(loc=both_dist_mean, scale=both_dist_sd, size=(2, 30, 500))

ttest_vals = []

# Sig diff sample comparisons
for sample1, sample2 in zip(sig_diff_samples_0v10, sig_diff_samples_10v0):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)
    ttest_vals.append(ttest_ind(sample2, sample1).statistic)

for sample1, sample2 in zip(sig_diff_samples_0v100, sig_diff_samples_100v0):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)
    ttest_vals.append(ttest_ind(sample2, sample1).statistic)

for sample1, sample2 in zip(sig_diff_samples_0v1000, sig_diff_samples_1000v0):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)
    ttest_vals.append(ttest_ind(sample2, sample1).statistic)

for sample1, sample2 in zip(sig_diff_samples_0v10_same_std, sig_diff_samples_10v0_same_std):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)
    ttest_vals.append(ttest_ind(sample2, sample1).statistic)

for sample1, sample2 in zip(sig_diff_samples_0v100_same_std, sig_diff_samples_100v0_same_std):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)
    ttest_vals.append(ttest_ind(sample2, sample1).statistic)

for sample1, sample2 in zip(sig_diff_samples_0v1000_same_std, sig_diff_samples_1000v0_same_std):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)
    ttest_vals.append(ttest_ind(sample2, sample1).statistic)

# Null sample comparisons
for sample1, sample2 in zip(null_samples_0v10[0], null_samples_0v10[1]):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)

for sample1, sample2 in zip(null_samples_0v100[0], null_samples_0v100[1]):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)

for sample1, sample2 in zip(null_samples_0v1000[0], null_samples_0v1000[1]):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)

for sample1, sample2 in zip(null_samples_0v10_same_std[0], null_samples_0v10_same_std[1]):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)

for sample1, sample2 in zip(null_samples_0v100_same_std[0], null_samples_0v100_same_std[1]):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)

for sample1, sample2 in zip(null_samples_0v1000_same_std[0], null_samples_0v1000_same_std[1]):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)

# Permuted sample comparisons
for sample1, sample2 in zip(permuted_samples_0v10[0], permuted_samples_0v10[1]):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)

for sample1, sample2 in zip(permuted_samples_0v100[0], permuted_samples_0v100[1]):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)

for sample1, sample2 in zip(permuted_samples_0v1000[0], permuted_samples_0v1000[1]):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)

for sample1, sample2 in zip(permuted_samples_0v10_same_std[0], permuted_samples_0v10_same_std[1]):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)

for sample1, sample2 in zip(permuted_samples_0v100_same_std[0], permuted_samples_0v100_same_std[1]):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)

for sample1, sample2 in zip(permuted_samples_0v1000_same_std[0], permuted_samples_0v1000_same_std[1]):
    ttest_vals.append(ttest_ind(sample1, sample2).statistic)

ttest_vals = np.array(ttest_vals)

# GP tree: takes two arrays as input, returns a test staistic
pset = gp.PrimitiveSetTyped('MAIN', [np.ndarray, np.ndarray], float)
pset.renameArguments(ARG0='x1')
pset.renameArguments(ARG1='x2')

# Logical operators on the distance array
#pset.addPrimitive(np.logical_and, [np.ndarray, np.ndarray], np.ndarray, name='array_and')
#pset.addPrimitive(np.logical_or, [np.ndarray, np.ndarray], np.ndarray, name='array_or')
#pset.addPrimitive(np.logical_xor, [np.ndarray, np.ndarray], np.ndarray, name='array_xor')
#pset.addPrimitive(np.logical_not, [np.ndarray], np.ndarray, name='array_not')

# Mathematical operators on the distance array
#pset.addPrimitive(np.add, [np.ndarray, np.ndarray], np.ndarray, name='array_add')
#pset.addPrimitive(np.subtract, [np.ndarray, np.ndarray], np.ndarray, name='array_sub')
#pset.addPrimitive(np.multiply, [np.ndarray, np.ndarray], np.ndarray, name='array_mul')
#pset.addPrimitive(np.divide, [np.ndarray, np.ndarray], np.ndarray, name='array_div')
pset.addPrimitive(np.sqrt, [np.ndarray], np.ndarray, name='array_sqrt')
pset.addPrimitive(np.square, [np.ndarray], np.ndarray, name='array_square')
pset.addPrimitive(np.abs, [np.ndarray], np.ndarray, name='array_abs')

# Statistics derived from the distance array
pset.addPrimitive(np.mean, [np.ndarray], float, name='array_mean')
pset.addPrimitive(np.median, [np.ndarray], float, name='array_median')
pset.addPrimitive(np.min, [np.ndarray], float, name='array_min')
pset.addPrimitive(np.max, [np.ndarray], float, name='array_max')
pset.addPrimitive(np.std, [np.ndarray], float, name='array_std')
pset.addPrimitive(np.var, [np.ndarray], float, name='array_var')
pset.addPrimitive(np.size, [np.ndarray], float, name='array_size')
pset.addPrimitive(np.sum, [np.ndarray], float, name='array_sum')
pset.addPrimitive(scipy.stats.sem, [np.ndarray], float, name='array_stderr')

# Mathematical operators with single values
def protected_div(left, right):
    try:
        return float(left) / float(right)
    except ZeroDivisionError:
        return 1.

pset.addPrimitive(operator.add, [float, float], float, name='float_add')
pset.addPrimitive(operator.sub, [float, float], float, name='float_sub')
pset.addPrimitive(operator.mul, [float, float], float, name='float_mul')
pset.addPrimitive(protected_div, [float, float], float, name='float_div')
pset.addPrimitive(np.sqrt, [float], float, name='float_sqrt')
pset.addPrimitive(np.square, [float], float, name='float_square')
pset.addPrimitive(np.abs, [float], float, name='float_abs')

# Mathematical operators on the distance array with a single value
pset.addPrimitive(np.add, [np.ndarray, float], np.ndarray, name='array_add_float')
pset.addPrimitive(np.subtract, [np.ndarray, float], np.ndarray, name='array_sub_float')
pset.addPrimitive(np.multiply, [np.ndarray, float], np.ndarray, name='array_mul_float')
pset.addPrimitive(np.divide, [np.ndarray, float], np.ndarray, name='array_div_float')

# Equivalence operators on the distance array with a single value
#pset.addPrimitive(np.less, [np.ndarray, float], np.ndarray, name='array_less_than_float')
#pset.addPrimitive(np.equal, [np.ndarray, float], np.ndarray, name='array_equal_float')

# Terminals
pset.addTerminal(1.0, float)
#pset.addEphemeralConstant('rand{}'.format(np.random.randint(1e9)), lambda: np.random.random() * 100., float)
#pset.addTerminal(np.multiply(np.random.random(size=features.shape[0]), 100.), np.ndarray)
#pset.addTerminal(np.array([True] * features.shape[0]), np.ndarray)
#pset.addTerminal(np.array([False] * features.shape[0]), np.ndarray)

creator.create('FitnessMulti', base.Fitness, weights=tuple([-1.] * (num_splits + 1)))
#creator.create('FitnessMulti', base.Fitness, weights=tuple([-1.] * (ttest_vals.shape[0] * 2)))
creator.create('Individual', gp.PrimitiveTree, fitness=creator.FitnessMulti)

toolbox = base.Toolbox()
toolbox.register('expr', gp.genHalfAndHalf, pset=pset, min_=1, max_=4)

"""
def return_ttest():
    return creator.Individual.from_string('float_div(float_sub(array_mean(x1), array_mean(x2)), float_add(array_stderr(x1), array_stderr(x2)))', pset)
    #return creator.Individual.from_string('float_div(float_sub(array_mean(x1), array_mean(x2)), float_sqrt(float_add(float_div(array_var(x1), array_size(x1)), float_div(array_var(x2), array_size(x2)))))', pset)

toolbox.register('individual', return_ttest)
"""
toolbox.register('individual', tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)
toolbox.register('compile', gp.compile, pset=pset)

#pool = multiprocessing.Pool()
#toolbox.register('map', pool.map)

def evaluate_individual(individual):
    # If the solution contains a constant value in it, throw it out
    str_ind = str(individual)
    if '1.0' in str_ind:
        #return tuple([sys.maxsize] * (ttest_vals.shape[0] * 2))
        return tuple([sys.maxsize] * (num_splits + 1))
    
    try:
        func = toolbox.compile(expr=individual)
    except:
        #return tuple([sys.maxsize] * (ttest_vals.shape[0] * 2))
        return tuple([sys.maxsize] * (num_splits + 1))

    gp_ttest_vals = []
    
    # Sig diff sample comparisons
    for sample1, sample2 in zip(sig_diff_samples_0v10, sig_diff_samples_10v0):
        gp_ttest_vals.append(func(sample1, sample2))
        gp_ttest_vals.append(func(sample2, sample1))

    for sample1, sample2 in zip(sig_diff_samples_0v100, sig_diff_samples_100v0):
        gp_ttest_vals.append(func(sample1, sample2))
        gp_ttest_vals.append(func(sample2, sample1))

    for sample1, sample2 in zip(sig_diff_samples_0v1000, sig_diff_samples_1000v0):
        gp_ttest_vals.append(func(sample1, sample2))
        gp_ttest_vals.append(func(sample2, sample1))
    
    for sample1, sample2 in zip(sig_diff_samples_0v10_same_std, sig_diff_samples_10v0_same_std):
        gp_ttest_vals.append(func(sample1, sample2))
        gp_ttest_vals.append(func(sample2, sample1))

    for sample1, sample2 in zip(sig_diff_samples_0v100_same_std, sig_diff_samples_100v0_same_std):
        gp_ttest_vals.append(func(sample1, sample2))
        gp_ttest_vals.append(func(sample2, sample1))

    for sample1, sample2 in zip(sig_diff_samples_0v1000_same_std, sig_diff_samples_1000v0_same_std):
        gp_ttest_vals.append(func(sample1, sample2))
        gp_ttest_vals.append(func(sample2, sample1))

    # Null sample comparisons
    for sample1, sample2 in zip(null_samples_0v10[0], null_samples_0v10[1]):
        gp_ttest_vals.append(func(sample1, sample2))
        
    for sample1, sample2 in zip(null_samples_0v100[0], null_samples_0v100[1]):
        gp_ttest_vals.append(func(sample1, sample2))
        
    for sample1, sample2 in zip(null_samples_0v1000[0], null_samples_0v1000[1]):
        gp_ttest_vals.append(func(sample1, sample2))

    for sample1, sample2 in zip(null_samples_0v10_same_std[0], null_samples_0v10_same_std[1]):
        gp_ttest_vals.append(func(sample1, sample2))

    for sample1, sample2 in zip(null_samples_0v100_same_std[0], null_samples_0v100_same_std[1]):
        gp_ttest_vals.append(func(sample1, sample2))
        
    for sample1, sample2 in zip(null_samples_0v1000_same_std[0], null_samples_0v1000_same_std[1]):
        gp_ttest_vals.append(func(sample1, sample2))
    
    # Permuted sample comparisons
    for sample1, sample2 in zip(permuted_samples_0v10[0], permuted_samples_0v10[1]):
        gp_ttest_vals.append(func(sample1, sample2))
        
    for sample1, sample2 in zip(permuted_samples_0v100[0], permuted_samples_0v100[1]):
        gp_ttest_vals.append(func(sample1, sample2))
        
    for sample1, sample2 in zip(permuted_samples_0v1000[0], permuted_samples_0v1000[1]):
        gp_ttest_vals.append(func(sample1, sample2))
    
    for sample1, sample2 in zip(permuted_samples_0v10_same_std[0], permuted_samples_0v10_same_std[1]):
        gp_ttest_vals.append(func(sample1, sample2))
    
    for sample1, sample2 in zip(permuted_samples_0v100_same_std[0], permuted_samples_0v100_same_std[1]):
        gp_ttest_vals.append(func(sample1, sample2))
        
    for sample1, sample2 in zip(permuted_samples_0v1000_same_std[0], permuted_samples_0v1000_same_std[1]):
        gp_ttest_vals.append(func(sample1, sample2))
        
    gp_ttest_vals = np.array(gp_ttest_vals)

    # If the solution produces NaN or inf values, throw it out
    if np.any(np.isnan(gp_ttest_vals)) or np.any(np.isinf(gp_ttest_vals)):
        #return tuple([sys.maxsize] * (ttest_vals.shape[0] * 2))
        return tuple([sys.maxsize] * (num_splits + 1))

    # Third fitness component is the size (i.e., complexity) of the GP tree
    ind_complexity = np.sum([type(component) == gp.Primitive for component in individual])
    abs_diffs = np.abs(np.subtract(gp_ttest_vals, ttest_vals))
    
    return tuple([np.sum(abs_diffs[i::num_splits]) for i in range(num_splits)] + [ind_complexity])
    #return tuple(np.append(np.abs(np.subtract(gp_ttest_vals, ttest_vals)), [ind_complexity] * ttest_vals.shape[0]))

toolbox.register('evaluate', evaluate_individual)
toolbox.register('select', selAutomaticEpsilonLexicase)
toolbox.register('mate', gp.cxOnePoint)
toolbox.register('expr_mut', gp.genFull, min_=0, max_=3)
toolbox.register('mutate', gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

pop = toolbox.population(n=500)
#stats = tools.Statistics(lambda ind: np.sum(ind.fitness.values[:ttest_vals.shape[0]]))
stats = tools.Statistics(lambda ind: np.sum(ind.fitness.values[:-1]))
stats.register('avg', np.mean)
stats.register('std', np.std)
stats.register('min', np.min)
stats.register('max', np.max)

#t_test = creator.Individual.from_string('float_div(float_sub(array_mean(x1), array_mean(x2)), float_sqrt(float_add(float_div(array_var(x1), array_size(x1)), float_div(array_var(x2), array_size(x2)))))', pset)
#print('t-test fitness: {}'.format(evaluate_individual(t_test)))

#t_test = creator.Individual.from_string('float_div(float_sub(array_mean(x1), array_mean(x2)), float_add(array_stderr(x1), array_stderr(x2)))', pset)
#print('t-test fitness: {}'.format(evaluate_individual(t_test)))

pop, stats = algorithms.eaMuPlusLambda(population=pop, toolbox=toolbox,
                                       cxpb=0.5, mutpb=0.5, mu=500, lambda_=500,
                                       ngen=50, stats=stats)
print('')

pop_fitness = [(np.sum(ind.fitness.values[:-1]), ind.fitness.values[-1], str(ind)) for ind in pop]

for (fitness, size, ind) in list(sorted(pop_fitness))[:10]:
    print(fitness, size, ind)

<string>:1: RuntimeWarning: invalid value encountered in sqrt
/Users/randal_olson/anaconda/lib/python3.5/site-packages/numpy/lib/function_base.py:3569: RuntimeWarning: Invalid value encountered in median
  RuntimeWarning)


gen	nevals	avg        	std        	min   	max       
0  	500   	5.35513e+22	1.19308e+24	262157	2.6705e+25
1  	500   	1.77089e+18	1.97198e+19	262157	2.21361e+20
2  	500   	9.29716e+18	4.44026e+19	262157	2.21361e+20
3  	500   	1.37244e+19	5.33824e+19	262157	2.21361e+20
4  	500   	1.50525e+19	5.57267e+19	262157	2.21361e+20
5  	500   	2.03652e+19	6.39791e+19	262157	2.21361e+20
6  	500   	1.9037e+19 	6.20616e+19	262157	2.21361e+20
7  	500   	2.21361e+19	6.64083e+19	261620	2.21361e+20
8  	500   	2.34643e+19	6.81432e+19	261620	2.21361e+20
9  	500   	2.92196e+19	7.49286e+19	262157	2.21361e+20
10 	500   	3.14333e+19	7.72661e+19	262157	2.21361e+20
11 	500   	2.56779e+19	7.08853e+19	262157	2.21361e+20
12 	500   	2.65633e+19	7.19338e+19	262157	2.21361e+20
13 	500   	2.16934e+19	6.58138e+19	240117	2.21361e+20
14 	500   	2.03652e+19	6.39791e+19	240117	2.21361e+20
15 	500   	1.85943e+19	6.14028e+19	240117	2.21361e+20
16 	500   	2.47924e+19	6.98098e+19	240117	2.21361e+20
17 	500   	2.47924e+19	6.98098